In [43]:
import pandas as pd
from load_data import Balance, Instruments, AdvisedPortfolios, PriceDB, Singleton
from utils import get_current_pos, get_advised_port, get_recommendation

In [42]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
userid = 'A01'

In [4]:
old_port = get_current_pos(userid=userid)
new_port = get_advised_port(risk_profile=2)
new_port.loc[:, 'date'] = pd.to_datetime(new_port.date)

In [5]:
old_port

,date,itemcode,itemname,quantity,price,value,wt,tracking_code,exposure
0,2/11/2020 9:00:00 AM,C000001,현금,19565,1.00,19565.00,0.027733,CASH,CASH
1,2/11/2020 9:00:00 AM,D000001,예금,350000,1.01,353500.00,0.501074,DEPOSIT,DEPOSIT
2,2/11/2020 9:00:00 AM,A069500,KODEX 200,1,42936.90,42936.90,0.060862,K200,EQ_M_KR_LARGE
3,2/11/2020 9:00:00 AM,A360200,KINDEX 미국S&P500,2,10430.40,20860.80,0.029569,SPX,EQ_M_US_LARGE
4,2/11/2020 9:00:00 AM,A196230,KBSTAR 단기통안채,1,108498.00,108498.00,0.153792,BOK_D.4,MM
5,2/11/2020 9:00:00 AM,A302190,TIGER 중장기국채,1,51841.35,51841.35,0.073483,KTB_D5,FI_D_LONG
6,2/11/2020 9:00:00 AM,A329750,TIGER 미국달러단기채권액티브,8,9645.00,77160.00,0.109372,UST_D.5,AL_FX_USD
7,2/11/2020 9:00:00 AM,A367380,KINDEX 미국나스닥100,3,10374.00,31122.00,0.044114,NQ100,EQ_S_NASDAQ


In [6]:
new_port

,date,itemcode,wt,tracking_code,itemname,exposure
0,2021-02-26,A122260,0.250000,BOK_D1,KOSEF 통안채1년,FI_D_SHORT
1,2021-02-26,A130730,0.250000,MM,KOSEF 단기자금,MM
2,2021-02-26,A153130,0.217851,MM,KODEX 단기채권,MM
3,2021-02-26,A114260,0.152149,KTB_D3,KODEX 국고채3년,FI_D_SHORT
4,2021-02-26,A214980,0.100000,MM_PLUS,KODEX 단기채권PLUS,MM
5,2021-02-26,A102110,0.030000,K200,TIGER 200,EQ_M_KR_LARGE


In [14]:
pd.merge(old_port, new_port, left_on=['exposure'], right_on=['exposure'], how='outer', suffixes=['_old', '_new'])

,date_old,itemcode_old,itemname_old,quantity,price,value,wt_old,tracking_code_old,exposure,date_new,itemcode_new,wt_new,tracking_code_new,itemname_new
0,2/11/2020 9:00:00 AM,C000001,현금,19565.0,1.00,19565.00,0.027733,CASH,CASH,NaN,NaN,NaN,NaN,NaN
1,2/11/2020 9:00:00 AM,D000001,예금,350000.0,1.01,353500.00,0.501074,DEPOSIT,DEPOSIT,NaN,NaN,NaN,NaN,NaN
2,2/11/2020 9:00:00 AM,A069500,KODEX 200,1.0,42936.90,42936.90,0.060862,K200,EQ_M_KR_LARGE,2021-02-26,A102110,0.030000,K200,TIGER 200
3,2/11/2020 9:00:00 AM,A360200,KINDEX 미국S&P500,2.0,10430.40,20860.80,0.029569,SPX,EQ_M_US_LARGE,NaN,NaN,NaN,NaN,NaN
4,2/11/2020 9:00:00 AM,A196230,KBSTAR 단기통안채,1.0,108498.00,108498.00,0.153792,BOK_D.4,MM,2021-02-26,A130730,0.250000,MM,KOSEF 단기자금
5,2/11/2020 9:00:00 AM,A196230,KBSTAR 단기통안채,1.0,108498.00,108498.00,0.153792,BOK_D.4,MM,2021-02-26,A153130,0.217851,MM,KODEX 단기채권
6,2/11/2020 9:00:00 AM,A196230,KBSTAR 단기통안채,1.0,108498.00,108498.00,0.153792,BOK_D.4,MM,2021-02-26,A214980,0.100000,MM_PLUS,KODEX 단기채권PLUS
7,2/11/2020 9:00:00 AM,A302190,TIGER 중장기국채,1.0,51841.35,51841.35,0.073483,KTB_D5,FI_D_LONG,NaN,NaN,NaN,NaN,NaN
8,2/11/2020 9:00:00 AM,A329750,TIGER 미국달러단기채권액티브,8.0,9645.00,77160.00,0.109372,UST_D.5,AL_FX_USD,NaN,NaN,NaN,NaN,NaN
9,2/11/2020 9:00:00 AM,A367380,KINDEX 미국나스닥100,3.0,10374.00,31122.00,0.044114,NQ100,EQ_S_NASDAQ,NaN,NaN,NaN,NaN,NaN


In [11]:
new_port

,date,itemcode,wt,tracking_code,itemname,exposure
0,2021-02-26,A122260,0.250000,BOK_D1,KOSEF 통안채1년,FI_D_SHORT
1,2021-02-26,A130730,0.250000,MM,KOSEF 단기자금,MM
2,2021-02-26,A153130,0.217851,MM,KODEX 단기채권,MM
3,2021-02-26,A114260,0.152149,KTB_D3,KODEX 국고채3년,FI_D_SHORT
4,2021-02-26,A214980,0.100000,MM_PLUS,KODEX 단기채권PLUS,MM
5,2021-02-26,A102110,0.030000,K200,TIGER 200,EQ_M_KR_LARGE


In [23]:
# Assets in portfolio
# The price will be retrieved automatically
tickers = ["XBB.TO",   # iShares Core Canadian Universe Bond Index ETF
           "XIC.TO",   # iShares Core S&P/TSX Capped Composite Index ETF
           "ITOT",     # iShares Core S&P Total U.S. Stock Market ETF
           "IEFA",     # iShares Core MSCI EAFE ETF
           "IEMG"]     # iShares Core MSCI Emerging Markets ETF
quantities = [36, 64, 32, 8, 7]

In [31]:
class Asset:
    """
    Asset class.
    Holds the name, number of units, and the :class:`.Price` of the asset.
    """
    def __init__(self, ticker, quantity=0):
        """
        Initialization.
        Args:
            ticker (str): Ticker of the asset.
            quantity (int, optional): Number of units of the asset. Default is zero.
        """

        assert ticker is not None, "ticker symbol is a mandatory argument."
        assert isinstance(quantity, int), "quantity must be integer."

        self._ticker = ticker
        self._quantity = quantity

        # we set the price to ask
        self._price = Price(ticker_info["ask"], 'KRW')

    @property
    def quantity(self):
        """ (int): Number of units of the asset. """
        return self._quantity

    @quantity.setter
    def quantity(self, quantity):
        assert isinstance(quantity, int), "quantity must be integer."
        self._quantity = quantity

    @property
    def price(self):
        """ 
        (float): Price of the asset (in asset's own currency). 
        """
        return self._price.price

    def price_in(self, currency):
        """ 
        Price of the asset in specified currency. 
        Args:
            currency (str): Currency in which to obtain price of asset.
        """
        return self._price.price_in(currency)

    @property
    def currency(self):
        """ 
        (str): Currency of the asset. 
        """
        return self._price.currency

    @property
    def ticker(self):
        """
        (str): Ticker of the asset.
        """
        return self._ticker

    def market_value(self):
        """
        Computes the market value of the asset. 
        Returns:
            float: Market value of the asset (in asset's own currency).
        """
        return self.price * self._quantity

    def market_value_in(self, currency):
        """
        Computes the market value of the asset in specified currency. 
        Args:
            currency (str): Currency in which to obtain market value.
        Returns:
            float: Market value of the asset.
        """
        return self._price.price_in(currency) * self._quantity

    def buy(self, quantity, currency=None):
        """
        Buys (or sells) a specified amount of the asset.
        Args:
            quantity (int): If positive, it is the quantity to buy. If negative, it is the quantity to sell.
            currency (str, optional): Currency in which to obtain cost. Defaults to asset's own currency.
        Returns:
            (float): Cost of the units bought in specified ``currency``.
        """
        self._quantity += quantity
        if currency is None:
            return self._price.price * quantity
        
        return self._price.price_in(currency) * quantity

    def cost_of(self, units, currency=None):
        """
        Computes the cost to purchase the specified number of units.
        Args:
            units (int): Units interested in purchasing.
            currency (str, optional): Currency in which to convert the cost. Default is asset's own currency.
        Returns:
            (float): Cost of the purchase.
        """
        if currency is None:
            return self.price * units
      
        return self.price_in(currency) * units



In [35]:
class Price:
    """
    An instance of :class:`Price` holds a price and a currency.    
    """
    def __init__(self, price, currency="KRW"):
        """
        Initialization.
        Args:
            price (float): Price.
            currency (str, optional): Currency of price. Defaults to "KRW".
        """
        self._price = price
        self._currency = currency.upper()

    @property
    def price(self):
        """
        (float): Price (in own's currency).
        """
        return self._price

    @property
    def currency(self):
        """
        (str): Currency of price.
        """
        return self._currency

    def price_in(self, currency):
        """
        Converts price in specified currency.
        Args:
            currency (str): Currency in which to convert the price.
        Returns:
            (float): Price in specified currency.
        """
#         currency_exchange = Cash.currency_rates.get_rate(
#             self.currency, currency.upper())
        currency_exchange = 1

        return currency_exchange * self._price

In [36]:
def easy_add_assets(tickers, quantities):
    """
    An easy way to add multiple assets to portfolio.
    Args:
        tickers (Sequence[str]): Ticker of assets in portfolio.
        quantities (Sequence[float]): Quantities of respective assets in portfolio. Must be in the same order as ``tickers``.
    """

    assert len(tickers) == len(quantities), \
           "`names` and `quantities` must be of the same length."

    for ticker, quantity in zip(tickers, quantities):
        self._assets[ticker] = Asset(ticker, quantity)

In [40]:
new_port

,date,itemcode,wt,tracking_code,itemname,exposure
0,2021-02-26,A122260,0.250000,BOK_D1,KOSEF 통안채1년,FI_D_SHORT
1,2021-02-26,A130730,0.250000,MM,KOSEF 단기자금,MM
2,2021-02-26,A153130,0.217851,MM,KODEX 단기채권,MM
3,2021-02-26,A114260,0.152149,KTB_D3,KODEX 국고채3년,FI_D_SHORT
4,2021-02-26,A214980,0.100000,MM_PLUS,KODEX 단기채권PLUS,MM
5,2021-02-26,A102110,0.030000,K200,TIGER 200,EQ_M_KR_LARGE


In [44]:
price_db = PriceDB.instance().data

In [58]:
new_port.merge(price_db, left_on=['date', 'itemcode'], right_on=['date', 'itemcode'], how='left')

,date,itemcode,wt,tracking_code,itemname,exposure,price,volume,trading_amt_mln,ret,itemtype
0,2021-02-26,A122260,0.250000,BOK_D1,KOSEF 통안채1년,FI_D_SHORT,101155.0,829489.0,61672.391133,0.000000,ETF
1,2021-02-26,A130730,0.250000,MM,KOSEF 단기자금,MM,100960.0,965560.0,64369.329522,0.000000,ETF
2,2021-02-26,A153130,0.217851,MM,KODEX 단기채권,MM,102725.0,39030.0,2884.683672,-0.000049,ETF
3,2021-02-26,A114260,0.152149,KTB_D3,KODEX 국고채3년,FI_D_SHORT,58090.0,5062.0,821.950469,-0.001204,ETF
4,2021-02-26,A214980,0.100000,MM_PLUS,KODEX 단기채권PLUS,MM,102995.0,9495.0,80590.448314,0.000049,ETF
5,2021-02-26,A102110,0.030000,K200,TIGER 200,EQ_M_KR_LARGE,41540.0,1256046.0,77215.429155,-0.028127,ETF


In [55]:
new_port.loc[:, 'date'] = pd.to_datetime(new_port.date)

In [57]:
new_port.date

0   2021-02-26
1   2021-02-26
2   2021-02-26
3   2021-02-26
4   2021-02-26
5   2021-02-26
Name: date, dtype: datetime64[ns]

In [56]:
price_db.date

0       2016-08-17
1       2016-08-18
2       2016-08-19
3       2016-08-22
4       2016-08-23
           ...    
24613   2021-03-04
24614   2021-03-05
24615   2021-03-08
24616   2021-03-09
24617   2021-03-10
Name: date, Length: 24618, dtype: datetime64[ns]

In [37]:
old_alloc = easy_add_assets(tickers=tickers, quantities=quantities)

NameError: name 'ticker_info' is not defined

In [22]:
new_port.loc[:, ['exposure', 'wt']].to_dict('list')

{'exposure': ['FI_D_SHORT', 'MM', 'MM', 'FI_D_SHORT', 'MM', 'EQ_M_KR_LARGE'],
 'wt': [0.2499999999983805,
  0.24999999995716302,
  0.21785129484296648,
  0.1521487051029433,
  0.10000000000386107,
  0.029999999987254864]}

In [ ]:
def rebalance(self, target_allocation, verbose=False):
    """
    Rebalances the portfolio using the specified target allocation, the portfolio's current allocation,
    and the available cash.
    Args:
        target_allocation (Dict[str, float]): Target asset allocation of the portfolio (in %). The keys of the dictionary are the tickers of the assets.
        verbose (bool, optional): Verbosity flag. Default is False. 
    Returns:
        (tuple): tuple containing:
            * new_units (Dict[str, int]): Units of each asset to buy. The keys of the dictionary are the tickers of the assets.
            * prices (Dict[str, [float, str]]): The keys of the dictionary are the tickers of the assets. Each value of the dictionary is a 2-entry list. The first entry is the price of the asset during the rebalancing computation. The second entry is the currency of the asset.
            * exchange_rates (Dict[str, float]): The keys of the dictionary are currencies. Each value is the exchange rate to CAD during the rebalancing computation.
            * max_diff (float): Largest difference between target allocation and optimized asset allocation.
    """

    # order target_allocation dict in the same order as assets dict and upper key
    target_allocation_reordered = {}
    try:
        for key in self.assets:
            target_allocation_reordered[key] = target_allocation[key]
    except:
        raise Exception(
            "'target_allocation not compatible with the assets of the portfolio."
        )

    target_allocation_np = np.fromiter(
        target_allocation_reordered.values(), dtype=float)

    assert abs(np.sum(target_allocation_np) -
               100.) <= 1E-2, "target allocation must sum up to 100%."

    # offload heavy work
    (balanced_portfolio, new_units, prices, cost, exchange_history) = rebalancing_helper.rebalance(self, target_allocation_np)

    # compute old and new asset allocation
    # and largest diff between new and target asset allocation
    old_alloc = self.asset_allocation()
    new_alloc = balanced_portfolio.asset_allocation()
    max_diff = max(
        abs(target_allocation_np -
            np.fromiter(new_alloc.values(), dtype=float)))

    if verbose:
        print("")
        # Print shares to buy, cost, new allocation, old allocation target, and target allocation
        print(
            " Ticker      Ask     Quantity      Amount    Currency     Old allocation   New allocation     Target allocation"
        )
        print(
            "                      to buy         ($)                      (%)              (%)                 (%)"
        )
        print(
            "---------------------------------------------------------------------------------------------------------------"
        )
        for ticker in balanced_portfolio.assets:
            print("%8s  %7.2f   %6.d        %8.2f     %4s          %5.2f            %5.2f               %5.2f" % \
            (ticker, prices[ticker][0], new_units[ticker], cost[ticker], prices[ticker][1], \
             old_alloc[ticker], new_alloc[ticker], target_allocation[ticker]))

        print("")
        print(
            "Largest discrepancy between the new and the target asset allocation is %.2f %%."
            % (max_diff))

        # Print conversion exchange
        if len(exchange_history) > 0:
            print("")
            if len(exchange_history) > 1:
                print(
                    "Before making the above purchases, the following currency conversions are required:"
                )
            else:
                print(
                    "Before making the above purchases, the following currency conversion is required:"
                )

            for exchange in exchange_history:
                (from_amount, from_currency, to_amount, to_currency,
                 rate) = exchange
                print("    %.2f %s to %.2f %s at a rate of %.4f." %
                      (from_amount, from_currency, to_amount, to_currency,
                       rate))

        # Print remaining cash
        print("")
        print("Remaining cash:")
        for cash in balanced_portfolio.cash.values():
            print("    %.2f %s." % (cash.amount, cash.currency))

    # Now that we're done, we can replace old portfolio with the new one
    self.__dict__.update(balanced_portfolio.__dict__)

    return (new_units, prices, exchange_history, max_diff)

In [6]:
rootpath = '../../'
rel_path = './data/processed/'
filename = 'pa_weights.pkl'

In [7]:
weights = pd.read_pickle(rootpath+rel_path+filename)

In [8]:
rebal_date = '2021-02-26'  # 종목 가격 입수시점(현재는 종가 기준)

In [10]:
filename = 'balance_s.pkl'
balance_s = pd.read_pickle(rootpath+rel_path+filename)

In [125]:
balance_s

,date,userid,name,asset_class,itemcode,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,투자원본
0,1/10/2020 1:00:00 PM,A01,투자자1,현금성,C000001,현금,700000,1,700000,1.00,700000.00,1.000000,20200110 13:00현금성,Y
1,1/10/2020 4:00:00 PM,A01,투자자1,현금성,C000001,현금,11055,1,11055,1.00,11055.00,0.015747,20200110 16:00현금성,N
2,1/10/2020 4:00:00 PM,A01,투자자1,현금성,D000001,예금,350000,1,350000,1.01,353500.00,0.503518,20200110 16:00현금성,N
3,1/10/2020 4:00:00 PM,A01,투자자1,주식,A069500,KODEX 200,1,42095,42095,41253.10,41253.10,0.058760,20200110 16:00주식,N
4,1/10/2020 4:00:00 PM,A01,투자자1,주식,A360200,KINDEX 미국S&P500,4,10865,43460,11082.30,44329.20,0.063142,20200110 16:00주식,N
5,1/10/2020 4:00:00 PM,A01,투자자1,채권,A196230,KBSTAR 단기통안채,1,104325,104325,100152.00,100152.00,0.142654,20200110 16:00채권,N
6,1/10/2020 4:00:00 PM,A01,투자자1,채권,A302190,TIGER 중장기국채,1,52365,52365,52365.00,52365.00,0.074588,20200110 16:00채권,N
7,1/10/2020 4:00:00 PM,A01,투자자1,대체,A329750,TIGER 미국달러단기채권액티브,4,9645,38580,10030.80,40123.20,0.057151,20200110 16:00대체,N
8,1/10/2020 4:00:00 PM,A01,투자자1,채권,A114260,KODEX 국고채3년,1,58120,58120,59282.40,59282.40,0.084441,20200110 16:00채권,N
9,2/11/2020 9:00:00 AM,A01,투자자1,현금성,C000001,현금,19565,1,19565,1.00,19565.00,0.027745,20200211 09:00현금성,N


In [11]:
# balance는 사용자(userid)의 최근 잔고내역.
balance = balance_s.loc[(balance_s.userid==userid) & (balance_s.date==balance_s.groupby(by='userid')['date'].max()[userid])]

In [12]:
balance

,date,userid,name,asset_class,itemcode,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,투자원본
9,2/11/2020 9:00:00 AM,A01,투자자1,현금성,C000001,현금,19565,1,19565,1.00,19565.00,0.027745,20200211 09:00현금성,N
10,2/11/2020 9:00:00 AM,A01,투자자1,현금성,D000001,예금,350000,1,350000,1.01,353500.00,0.501298,20200211 09:00현금성,N
11,2/11/2020 9:00:00 AM,A01,투자자1,주식,A069500,KODEX 200,1,42095,42095,39990.25,39990.25,0.056710,20200211 09:00주식,N
12,2/11/2020 9:00:00 AM,A01,투자자1,주식,A360200,KINDEX 미국S&P500,2,10865,21730,10973.65,21947.30,0.031123,20200211 09:00주식,N
13,2/11/2020 9:00:00 AM,A01,투자자1,채권,A196230,KBSTAR 단기통안채,1,104325,104325,109541.25,109541.25,0.155340,20200211 09:00채권,N
14,2/11/2020 9:00:00 AM,A01,투자자1,채권,A302190,TIGER 중장기국채,1,52365,52365,52365.00,52365.00,0.074259,20200211 09:00채권,N
15,2/11/2020 9:00:00 AM,A01,투자자1,대체,A329750,TIGER 미국달러단기채권액티브,8,9645,77160,9355.65,74845.20,0.106138,20200211 09:00대체,N
16,2/11/2020 9:00:00 AM,A01,투자자1,주식,A367380,KINDEX 미국나스닥100,3,10920,32760,11138.40,33415.20,0.047386,20200211 09:00주식,N


In [85]:
old_port = balance.loc[:, ['itemcode', 'itemname', 'wt', 'value', 'quantity', 'price']]

In [100]:
access_path='../../data/processed/'
filename='advised_portfolios.pkl'
advised_pf = pd.read_pickle(access_path+filename)

In [83]:
from datetime import timedelta, datetime

In [84]:
balance_s.loc[(balance_s.userid==userid) & (balance_s.date==balance_s.groupby(by='userid')['date'].max()[userid])]

,date,userid,name,asset_class,itemcode,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,투자원본
9,2/11/2020 9:00:00 AM,A01,투자자1,현금성,C000001,현금,19565,1,19565,1.00,19565.00,0.027745,20200211 09:00현금성,N
10,2/11/2020 9:00:00 AM,A01,투자자1,현금성,D000001,예금,350000,1,350000,1.01,353500.00,0.501298,20200211 09:00현금성,N
11,2/11/2020 9:00:00 AM,A01,투자자1,주식,A069500,KODEX 200,1,42095,42095,39990.25,39990.25,0.056710,20200211 09:00주식,N
12,2/11/2020 9:00:00 AM,A01,투자자1,주식,A360200,KINDEX 미국S&P500,2,10865,21730,10973.65,21947.30,0.031123,20200211 09:00주식,N
13,2/11/2020 9:00:00 AM,A01,투자자1,채권,A196230,KBSTAR 단기통안채,1,104325,104325,109541.25,109541.25,0.155340,20200211 09:00채권,N
14,2/11/2020 9:00:00 AM,A01,투자자1,채권,A302190,TIGER 중장기국채,1,52365,52365,52365.00,52365.00,0.074259,20200211 09:00채권,N
15,2/11/2020 9:00:00 AM,A01,투자자1,대체,A329750,TIGER 미국달러단기채권액티브,8,9645,77160,9355.65,74845.20,0.106138,20200211 09:00대체,N
16,2/11/2020 9:00:00 AM,A01,투자자1,주식,A367380,KINDEX 미국나스닥100,3,10920,32760,11138.40,33415.20,0.047386,20200211 09:00주식,N


In [73]:
risk_profile=2

In [86]:
new_port = advised_pf.loc[(advised_pf.risk_profile==risk_profile) & (advised_pf.date==advised_pf.groupby(by='risk_profile')['date'].max()[risk_profile])]

In [96]:
new_port = new_port.loc[:, ['itemcode', 'itemname', 'weights']]

In [97]:
pd.merge(old_port, new_port, left_on=['itemcode', 'itemname'], right_on=['itemcode', 'itemname'], how='outer')

,itemcode,itemname,wt,value,quantity,price,weights
0,C000001,현금,0.027745,19565.00,19565.0,1.00,NaN
1,D000001,예금,0.501298,353500.00,350000.0,1.01,NaN
2,A069500,KODEX 200,0.056710,39990.25,1.0,39990.25,NaN
3,A360200,KINDEX 미국S&P500,0.031123,21947.30,2.0,10973.65,NaN
4,A196230,KBSTAR 단기통안채,0.155340,109541.25,1.0,109541.25,NaN
5,A302190,TIGER 중장기국채,0.074259,52365.00,1.0,52365.00,NaN
6,A329750,TIGER 미국달러단기채권액티브,0.106138,74845.20,8.0,9355.65,NaN
7,A367380,KINDEX 미국나스닥100,0.047386,33415.20,3.0,11138.40,NaN
8,A122260,KOSEF 통안채1년,NaN,NaN,NaN,NaN,0.250000
9,A130730,KOSEF 단기자금,NaN,NaN,NaN,NaN,0.250000


In [102]:
new_port = new_port.rename(columns={'weights':'wt'})

In [103]:
new_port

,itemcode,itemname,wt
16051,A122260,KOSEF 통안채1년,0.250000
16052,A130730,KOSEF 단기자금,0.250000
16053,A153130,KODEX 단기채권,0.217851
16054,A114260,KODEX 국고채3년,0.152149
16055,A214980,KODEX 단기채권PLUS,0.100000
16056,A102110,TIGER 200,0.030000


In [104]:
pd.merge(old_port, new_port, left_on=['itemcode', 'itemname'], right_on=['itemcode', 'itemname'], how='outer', suffixes=['_old', '_new'])

,itemcode,itemname,wt_old,value,quantity,price,wt_new
0,C000001,현금,0.027745,19565.00,19565.0,1.00,NaN
1,D000001,예금,0.501298,353500.00,350000.0,1.01,NaN
2,A069500,KODEX 200,0.056710,39990.25,1.0,39990.25,NaN
3,A360200,KINDEX 미국S&P500,0.031123,21947.30,2.0,10973.65,NaN
4,A196230,KBSTAR 단기통안채,0.155340,109541.25,1.0,109541.25,NaN
5,A302190,TIGER 중장기국채,0.074259,52365.00,1.0,52365.00,NaN
6,A329750,TIGER 미국달러단기채권액티브,0.106138,74845.20,8.0,9355.65,NaN
7,A367380,KINDEX 미국나스닥100,0.047386,33415.20,3.0,11138.40,NaN
8,A122260,KOSEF 통안채1년,NaN,NaN,NaN,NaN,0.250000
9,A130730,KOSEF 단기자금,NaN,NaN,NaN,NaN,0.250000


In [7]:
from client import Client


NameError: name 'Singleton' is not defined